In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        
        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        for i in model_list:
            print(i)
            model = load_model(i)
            cur_pred = model.predict_generator(testing_generator, verbose = 1)
            cur_pred_thres = []
            pred_list = []
            
            for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
            prediction_array.append(pred_list)
            
            
            y_pred = np.array(cur_pred_thres)
            y_true = np.array(list(subject_dict_test.values()))
            acc = accuracy_score(y_pred, y_true)
            conf = confusion_matrix(y_pred, y_true)
            sen = conf[0,0]/(conf[0,0] + conf[1,0])
            spec = conf[1,1]/(conf[1,1] + conf[0,1])
            f1 = f1_score(y_pred, y_true)
            
            print('sen',sen)
            print('spec', spec)
            print('f1',f1)
            
            accuracy_list.append(acc)
            confusion_matrix_list.append(conf)
            sensitivity_list.append(sen)
            specificity_list.append(spec)
            f1_score_list.append(f1)
            
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_MCI_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models'
model_list = ['top_model_1.hdf5','top_model_2.hdf5','top_model_3.hdf5','top_model_4.hdf5','top_model_5.hdf5','top_model_6.hdf5'
             ,'top_model_7.hdf5','top_model_8.hdf5','top_model_9.hdf5']
full_model_list = []

for i in model_list:
    full_model_list.append(model_list_path + '/' + i)
    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data(full_model_list, cv_file, params_dict)


/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models/top_model_1.hdf5
Instructions for updating:
Please use Model.predict, which supports generators.
3744/3744 [==============================] - 56s 15ms/step
sen 0.917157440125638
spec 0.6215538847117794
f1 0.6914498141263941
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models/top_model_2.hdf5
3744/3744 [==============================] - 19s 5ms/step
sen 0.9289360031409502
spec 0.6131996658312447
f1 0.6950757575757576
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models/top_model_3.hdf5
3744/3744 [==============================] - 20s 5ms/step
sen 0.8849627012171182
spec 0.6741854636591479
f1 0.7026556377884198
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/final_models/top_model_4.hdf5
3744/3744 [==============================] - 22s 6ms/step
sen 0.8716136631330977
spec 0.66416040100

In [2]:
result_list[1]

[0.8226495726495726,
 0.8279914529914529,
 0.8175747863247863,
 0.8052884615384616,
 0.8052884615384616,
 0.7996794871794872,
 0.7889957264957265,
 0.8181089743589743,
 0.8210470085470085]

model averaging

In [3]:
p = result_list[0]

In [4]:
a = np.array(p)
a.shape

(9, 3744)

In [5]:
aT = a.T

In [6]:
aT.shape

(3744, 9)

top-3

In [7]:
aT3 = aT[:, 0:3]

In [8]:
aT3.shape

(3744, 3)

In [9]:
p_avg = np.average(aT3, axis = 1)

In [10]:
p_avg.shape

(3744,)

In [11]:
super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

In [12]:
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
        
        
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




In [13]:
accuracy_score(p_avg, np.array(list(subject_dict_test.values())))

0.8239850427350427

In [14]:
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))

In [15]:
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

sen 0.9089124460149195
spec 0.6432748538011696


In [16]:
f1_score(p_avg, np.array(list(subject_dict_test.values())))

0.700318326512051

top-5


In [17]:
aT5 = aT[:, 0:5]
print(aT5.shape)
p_avg = np.average(aT5, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(3744, 5)
(3744,)
acc 0.8213141025641025
sen 0.8924224577934825
spec 0.6700083542188805


0.7056753189617246

top-7

In [18]:
aT7 = aT[:, 0:7]
print(aT7.shape)
p_avg = np.average(aT7, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(3744, 7)
(3744,)
acc 0.8258547008547008
sen 0.9073419709462113
spec 0.6524644945697577


0.7055103884372177

top-9

In [19]:
aT9 = aT[:, 0:9]
print(aT9.shape)
p_avg = np.average(aT9, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(3744, 9)
(3744,)
acc 0.8311965811965812
sen 0.9191205339615234
spec 0.6441102756892231


0.7092916283348666

top-4

In [20]:
aT4 = aT[:, 0:4]
print(aT4.shape)
p_avg = np.average(aT4, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(3744, 4)
(3744,)
acc 0.8245192307692307
sen 0.9038084020416176
spec 0.6558061821219716


0.7049842837898518

top-6

In [21]:
aT6 = aT[:, 0:6]
print(aT6.shape)
p_avg = np.average(aT6, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(3744, 6)
(3744,)
acc 0.8269230769230769
sen 0.911660777385159
spec 0.6466165413533834


0.7049180327868851

top-8

In [22]:
aT8 = aT[:, 0:8]
print(aT8.shape)
p_avg = np.average(aT8, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(3744, 8)
(3744,)
acc 0.8282585470085471
sen 0.9148017275225756
spec 0.6441102756892231


0.7057208237986271

max voting

In [23]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        
        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        for i in model_list:
            print(i)
            model = load_model(i)
            cur_pred = model.predict_generator(testing_generator, verbose = 1)
            cur_pred_thres = []
            pred_list = []
            
            for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
            prediction_array.append(pred_list)
            
            
            y_pred = np.array(cur_pred_thres)
            y_true = np.array(list(subject_dict_test.values()))
            acc = accuracy_score(y_pred, y_true)
            conf = confusion_matrix(y_pred, y_true)
            sen = conf[0,0]/(conf[0,0] + conf[1,0])
            spec = conf[1,1]/(conf[1,1] + conf[0,1])
            f1 = f1_score(y_pred, y_true)
            
            print('sen',sen)
            print('spec', spec)
            print('f1',f1)
            
            accuracy_list.append(acc)
            confusion_matrix_list.append(conf)
            sensitivity_list.append(sen)
            specificity_list.append(spec)
            f1_score_list.append(f1)
            
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/MCI_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_MCI_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models'
model_list = ['top_model_1.hdf5','top_model_2.hdf5','top_model_3.hdf5','top_model_4.hdf5','top_model_5.hdf5','top_model_6.hdf5'
             ,'top_model_7.hdf5','top_model_8.hdf5','top_model_9.hdf5']
full_model_list = []

for i in model_list:
    full_model_list.append(model_list_path + '/' + i)
    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data(full_model_list, cv_file, params_dict)


/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_1.hdf5
3744/3744 [==============================] - 19s 5ms/step
sen 0.5394581861012956
spec 0.8170426065162907
f1 0.5842293906810035
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_2.hdf5
3744/3744 [==============================] - 19s 5ms/step
sen 0.6230859835100118
spec 0.7644110275689223
f1 0.5957031250000001
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_3.hdf5
3744/3744 [==============================] - 19s 5ms/step
sen 0.48095798979191207
spec 0.8170426065162907
f1 0.5593365742064627
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_4.hdf5
3744/3744 [==============================] - 21s 6ms/step
sen 0.574401256380055
spec 0.7301587301587301
f1 0.554041204437401
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Basel

top-3

In [24]:
p = result_list[0]
a = np.array(p)
a.shape

(9, 3744)

In [25]:
aT = a.T
aT.shape

(3744, 9)

In [26]:
aT3 = aT[:, 0:3]
print(aT3.shape)

super_threshold_indices = aT3  > 0.5
aT3[super_threshold_indices] = 1
super_threshold_indices = aT3  <= 0.5
aT3[super_threshold_indices] = 0



(3744, 3)


In [27]:
nz_count = np.count_nonzero(aT3, axis = 1)

In [28]:
y_pred = []
for nzc in nz_count:
    if nzc >= 2:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [29]:
y_pred_arr = np.array(y_pred)

In [30]:
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y


In [31]:

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))

acc 0.6380876068376068
sen 0.563800549666274
spec 0.7961570593149541


0.5844832873351733

top-5

In [35]:
aT5 = aT[:, 0:5]
print(aT5.shape)

super_threshold_indices = aT5  > 0.5
aT5[super_threshold_indices] = 1
super_threshold_indices = aT5  <= 0.5
aT5[super_threshold_indices] = 0

nz_count = np.count_nonzero(aT5, axis = 1)
y_pred = []
for nzc in nz_count:
    if nzc >= 3:
        y_pred.append(1)
    else:
        y_pred.append(0)

y_pred_arr = np.array(y_pred)
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))
f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))


(3744, 5)
acc 0.6276709401709402
sen 0.5484884177463683
spec 0.7961570593149541


0.5775757575757575

top-7

In [36]:
aT7 = aT[:, 0:7]
print(aT5.shape)

super_threshold_indices = aT7  > 0.5
aT7[super_threshold_indices] = 1
super_threshold_indices = aT7  <= 0.5
aT7[super_threshold_indices] = 0

nz_count = np.count_nonzero(aT7, axis = 1)
y_pred = []
for nzc in nz_count:
    if nzc >= 4:
        y_pred.append(1)
    else:
        y_pred.append(0)

y_pred_arr = np.array(y_pred)
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))
f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))


(3744, 5)
acc 0.6319444444444444
sen 0.5555555555555556
spec 0.7944862155388471


0.5798780487804879

top-9

In [37]:
aT9 = aT[:, 0:9]
print(aT9.shape)

super_threshold_indices = aT9  > 0.5
aT9[super_threshold_indices] = 1
super_threshold_indices = aT9  <= 0.5
aT9[super_threshold_indices] = 0

nz_count = np.count_nonzero(aT9, axis = 1)
y_pred = []
for nzc in nz_count:
    if nzc >= 5:
        y_pred.append(1)
    else:
        y_pred.append(0)

y_pred_arr = np.array(y_pred)
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))
f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))


(3744, 9)
acc 0.6396901709401709
sen 0.5716529250098155
spec 0.7844611528822055


0.5819646730709637